## 深い研究

古典的なクロスビジネスエージェントのユースケースの1つ！これは巨大です。

<テーブルスタイル= "マージン：0;テキストアライグ：左;幅：100％">
    <tr>
        <td style = "width：150px; height：150px; vertical-align：middle;">
            <img src = "../ assets/business.png" width = "150" height = "150" style = "display：block;" />
        </td>
        <td>
            <h2 style = "color：＃00bfff;">商業的影響</h2>
            <SPAN STYLE = "Color：＃00BFFF;">深い研究エージェントは、あらゆるビジネス分野や日々の活動に広く適用できます。あなたはこれを自分で利用することができます！
            </span>
        </td>
    </tr>
</table>

In [29]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [ ]:
load_dotenv(override=True)

## Openaiホストツール

Openai Agents SDKには、次のホストツールが含まれています。

「WebearchTool」を使用すると、エージェントがWebを検索できます。  
`filesearchTool`を使用すると、OpenAIベクターストアから情報を取得できます。  
「ComputerTool」は、スクリーンショットの撮影やクリックなどのタスクを自動化することができます。

###重要なメモ-WebSearchToolのAPIチャージ

これには、Openai WebSearchToolの呼び出しごとに2.5セントの費用がかかります。次の2つのラボでは、最大2〜3ドルになります。他のプラットフォームで無料および低コストの検索ツールを使用するので、コストが懸念される場合は、これを実行して自由にスキップしてください。また、学生のクリスチャンW.は、Openaiが1回のコールの複数の検索に対して請求する場合があるため、通話ごとに2.5セント以上の費用がかかる場合があると指摘しました。

コストはこちら：https：//platform.openai.com/docs/pricing#web-searchです

In [31]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [ ]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

### いつものように、トレースを見てください

https://platform.openai.com/traces

### 構造化された出力を使用し、フィールドの説明を含めます

In [33]:
# WebSearchToolのコストに関する上記のメモを参照してください

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Pydanticを使用して、私たちの応答のスキーマを定義します - これは「構造化された出力」として知られています
# 学生のWes C.に大いに感謝して、これで厄介なバグを発見して修正してくれました！

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [ ]:

message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

In [35]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com") # これを確認したメールに変更します
    to_email = To("ed.donner@gmail.com") # これをメールに変更します
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [ ]:
send_email

In [37]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)



In [38]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

### 次の3つの関数は、planner_agentとsearch_agentを使用して、検索を計画して実行します

In [39]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("検索の計画...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("検索...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("検索が終了しました")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### 次の2つの関数はレポートを書き、それを電子メールで送信します

In [40]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("レポートについて考えています...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("レポートを書き終えました")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("メールを書く...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("電子メールが送信されました")
    return report

### ショータイム！

In [ ]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("研究を開始...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("hooray！")




### いつものように、トレースを見てください

https://platform.openai.com/traces

<テーブルスタイル= "マージン：0;テキストアライグ：左;幅：100％">
    <tr>
        <td style = "width：150px; height：150px; vertical-align：middle;">
            <img src = "../ assets/ants.png" width = "150" height = "150" style = "display：block;" />
        </td>
        <td>
            <h2 style = "color：＃00cc00;">あなたの進捗状況とリクエストのおめでとう</h2>
            <Span style = "color：＃00cc00;">コースで重要な瞬間に到達しました。最新のエージェントフレームワークの1つを使用して貴重なエージェントを作成しました。あなたはアップスキルし、新しい商業的可能性を解き放ちました。あなたの成功を祝うために少し時間を取ってください！あなたがウデミーでコースを評価できるなら、私は真剣に感謝するでしょう：それはウデミーが他の人にコースを示すかどうかを決定する最も重要な方法であり、それは大きな違いを生む。コースでの進捗状況について投稿したい場合は、タグを付けてください。露出を増やすために体重を計ります。
            </span>
        </td>
    </tr>